#### ML for classification - K-Nearest Neighbours 
- Unlike logistic regression, the kNN method has no model formation, and thus we cannot interpret the effects of features from the classifier.
- To classify a new observation $x$, the euclidean distance from $x$ to every point in the training set is measured. The k smallest distances are identified, and the **most represented class** by these k nearest neighbours is considered the output class label of $x$.  
![kNN.PNG](attachment:kNN.PNG)  
- Note that for a dataset with a mixture of numerical and categorical variables, the distance is only computed for variables that are numerical.

![knn_classifier.png](attachment:knn_classifier.png)

![knn_classifier2.png](attachment:knn_classifier2.png)

In [ ]:
install.packages('class')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [ ]:
library(dplyr)
library(class)  


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



#### Comparison of model performance (Logistic regression vs kNN)

In [ ]:
data(BreastCancer,package="mlbench") 
bc <- BreastCancer[complete.cases(BreastCancer),]
bc[,2:4] <- sapply(bc[,2:4], as.numeric) 
bc <- bc %>% mutate(y=factor(ifelse(Class=="malignant", 1,0))) %>% select(Cl.thickness:Cell.shape, y)

# Normalize numeric variables (Important when using kNN)
nor <- function(x) { 
    (x-min(x))/(max(x)-min(x)) 
} 
bc[,1:3] <- sapply(bc[,1:3], nor)

# Split data into training and test sets
set.seed(100) 
training.idx <- sample(nrow(bc), size=nrow(bc)*0.8) 
train.data <- bc[training.idx, ] 
test.data <- bc[-training.idx, ]

# kNN classification 
set.seed(101) 
knn1 <- knn(train.data[,1:3], test.data[,1:3], cl=train.data$y, k=2) 
mean(knn1==test.data$y)  # Accuracy of ~96%
table(knn1,test.data$y)

[1] 0.9562044

    
knn1  0  1
   0 89  5
   1  1 42

#### Loan application example:
- A data scientist in a bank was given a bank credit dataset and asked by his/her manager the following questions: Should the bank give a loan to an individual? Would an individual default on his/her loan? Is that person closer in characteristics to people who defaulted or did not default on their loans?

In [ ]:
gc <- read.csv('germancredit.csv', header=T); head(gc, 6)
str(gc)

Default,checkingstatus1,duration,history,purpose,amount,savings,employ,installment,status,...,residence,property,age,otherplans,housing,cards,job,liable,tele,foreign
<int>,<fct>,<int>,<fct>,<fct>,<int>,<fct>,<fct>,<int>,<fct>,...,<int>,<fct>,<int>,<fct>,<fct>,<int>,<fct>,<int>,<fct>,<fct>
0,A11,6,A34,A43,1169,A65,A75,4,A93,...,4,A121,67,A143,A152,2,A173,1,A192,A201
1,A12,48,A32,A43,5951,A61,A73,2,A92,...,2,A121,22,A143,A152,1,A173,1,A191,A201
0,A14,12,A34,A46,2096,A61,A74,2,A93,...,3,A121,49,A143,A152,1,A172,2,A191,A201
0,A11,42,A32,A42,7882,A61,A74,2,A93,...,4,A122,45,A143,A153,1,A173,2,A191,A201
1,A11,24,A33,A40,4870,A61,A73,3,A93,...,4,A124,53,A143,A153,2,A173,2,A191,A201
0,A14,36,A32,A46,9055,A65,A73,2,A93,...,4,A124,35,A143,A153,1,A172,2,A192,A201


'data.frame':	1000 obs. of  21 variables:
 $ Default        : int  0 1 0 0 1 0 0 0 0 1 ...
 $ checkingstatus1: Factor w/ 4 levels "A11","A12","A13",..: 1 2 4 1 1 4 4 2 4 2 ...
 $ duration       : int  6 48 12 42 24 36 24 36 12 30 ...
 $ history        : Factor w/ 5 levels "A30","A31","A32",..: 5 3 5 3 4 3 3 3 3 5 ...
 $ purpose        : Factor w/ 10 levels "A40","A41","A410",..: 5 5 8 4 1 8 4 2 5 1 ...
 $ amount         : int  1169 5951 2096 7882 4870 9055 2835 6948 3059 5234 ...
 $ savings        : Factor w/ 5 levels "A61","A62","A63",..: 5 1 1 1 1 5 3 1 4 1 ...
 $ employ         : Factor w/ 5 levels "A71","A72","A73",..: 5 3 4 4 3 3 5 3 4 1 ...
 $ installment    : int  4 2 2 2 3 2 3 2 2 4 ...
 $ status         : Factor w/ 4 levels "A91","A92","A93",..: 3 2 3 3 3 3 3 3 1 4 ...
 $ others         : Factor w/ 3 levels "A101","A102",..: 1 1 1 3 1 1 1 1 1 1 ...
 $ residence      : int  4 2 3 4 4 4 4 2 4 2 ...
 $ property       : Factor w/ 4 levels "A121","A122",..: 1 1 1 2 4 4 2 3 1 3 ...


In [ ]:
# Convert outcome variable (Default) to factor (categorical variable)
gc$Default <- factor(gc$Default)

# Find numeric variables
numvar <- sapply(gc, is.numeric); numvar  # sapply returns vector or matrix by default

# Normalize the numeric variables
nor <- function(x) {
    (x-min(x))/(max(x)-min(x))
}
gc[, numvar] <- sapply(gc[, numvar], nor)

# For a simple illustration, select only first 3 numeric variables 
gc1 <- gc %>% select("Default","duration", "amount", "installment") 

# Split data 
set.seed(100) 
training.idx <- sample(nrow(gc1), size=nrow(gc1)*0.8) 
train.data <- gc1[training.idx, ] 
test.data <- gc1[-training.idx, ]


Default checkingstatus1        duration         history         purpose 
          FALSE           FALSE            TRUE           FALSE           FALSE 
         amount         savings          employ     installment          status 
           TRUE           FALSE           FALSE            TRUE           FALSE 
         others       residence        property             age      otherplans 
          FALSE            TRUE           FALSE            TRUE           FALSE 
        housing           cards             job          liable            tele 
          FALSE            TRUE           FALSE            TRUE           FALSE 
        foreign 
          FALSE

In [ ]:
# Perform kNN classification across different k values
# Function knn() in package “class” is used to perform kNN classification with syntax below:
# knn(train, test, cl, k = 1, …), where the arguments “train” and “test” are matrices or dataframes of predictors 
# in the training and test sets (excluding the outcome variable). “cl” is the outcome variable of training set. 
# “k” is the number of neighbors considered (default=1).
acc <- rep(0, 30)  # List of accuracy reading for different k
for(i in 1:30) {
    set.seed(101)
    knn_cl <- knn(train.data[, 2:4], test.data[, 2:4], cl=train.data$Default, k=i)  # Exclude outcome variable
    acc[i] <- mean(knn_cl==test.data$Default)
    cat('k=', i, 'accuracy=', acc[i], '\n')
}

# Accuracy plot
plot(acc, type='b', xlab='K')  # best k=30


k= 1 accuracy= 0.58 
k= 2 accuracy= 0.565 
k= 3 accuracy= 0.625 
k= 4 accuracy= 0.65 
k= 5 accuracy= 0.635 
k= 6 accuracy= 0.655 
k= 7 accuracy= 0.65 
k= 8 accuracy= 0.655 
k= 9 accuracy= 0.67 
k= 10 accuracy= 0.685 
k= 11 accuracy= 0.68 
k= 12 accuracy= 0.705 
k= 13 accuracy= 0.73 
k= 14 accuracy= 0.725 
k= 15 accuracy= 0.74 
k= 16 accuracy= 0.73 
k= 17 accuracy= 0.725 
k= 18 accuracy= 0.71 
k= 19 accuracy= 0.73 
k= 20 accuracy= 0.725 
k= 21 accuracy= 0.71 
k= 22 accuracy= 0.72 
k= 23 accuracy= 0.73 
k= 24 accuracy= 0.725 
k= 25 accuracy= 0.73 
k= 26 accuracy= 0.75 
k= 27 accuracy= 0.75 
k= 28 accuracy= 0.76 
k= 29 accuracy= 0.75 
k= 30 accuracy= 0.76 


In [ ]:
# Perform kNN classification with optimal k
set.seed(100) 
knn2 <- knn(train.data[,2:4], test.data[,2:4], cl=train.data$Default, k=30) 
mean(knn2==test.data$Default)
# Confusion matrix
table(knn2,test.data$Default)

[1] 0.75

    
knn2   0   1
   0 138  30
   1  20  12

- Unlike the classification for breast cancer patients in the previous example, where the benign and malignant classes are well distinguished by those biological features, whether a loan applicant defaults on his/her loan or not is often affected by many other factors (might be qualitative), so the kNN classification accuracy is reasonably good in this example.